In [216]:
from collections import OrderedDict
from bs4 import BeautifulSoup
from IPython.display import Image
from itertools import cycle
from kmodes import kmodes
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords # Import the stop word list
from scipy import interp
#from sklearn import cross_validation, datasets, linear_model, metrics, svm, datasets, tree
from sklearn import datasets, linear_model, metrics, svm, datasets, tree
from sklearn.cluster import MeanShift
#from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals.six import StringIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, roc_curve, auc
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from statistics import mode, mean, median
import datetime
import graphviz as gv
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pydotplus
import re

import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols

In [217]:
df = pd.read_csv('admission_minogue-4.csv')
df

,Term,Level,Prospect,Applicant,Admit,Deposit,Enroll,Gender,BIRTHDATE,Ethnicity,...,PROG_ACTION,PROG_REASON,ApplicationDate,1stGeneration,Convert_GPA_Fix,FIN_AID_INTEREST,Rank,Size,Percentile,Pell
0,900,F,True,True,False,False,False,F,09-Jul-50,White,...,WADM,CANC,07-Oct-11,False,NaN,Y,NaN,NaN,NaN,False
1,900,F,True,True,False,False,False,U,28-Nov-88,Multiracial,...,DENY,INC,20-Apr-12,False,NaN,N,NaN,NaN,NaN,False
2,900,F,True,True,False,False,False,M,18-Jul-87,Hispanic,...,APPL,INC,14-Aug-12,False,NaN,N,NaN,NaN,NaN,False
3,900,F,True,True,False,False,False,M,27-Aug-93,White,...,DENY,INC,23-Nov-11,False,4.100,N,0.0,0.0,0.0,False
4,900,F,True,True,False,True,False,M,15-Jun-89,Hispanic,...,DENY,INC,06-Jul-12,True,NaN,Y,NaN,NaN,NaN,True
5,900,F,True,True,False,False,False,U,16-Dec-88,Asian/Pacific Islander,...,DENY,INC,21-Nov-11,False,NaN,Y,NaN,NaN,NaN,False
6,900,F,True,True,False,False,False,U,10-Jul-89,Hispanic,...,DENY,INC,30-Aug-11,False,2.538,N,0.0,0.0,0.0,False
7,900,F,True,True,False,False,False,M,28-May-90,White,...,DENY,COUR,27-Feb-12,False,2.730,N,0.0,0.0,0.0,False
8,900,F,True,True,False,False,False,F,19-Jul-70,Black,...,DENY,INC,26-Mar-12,False,NaN,Y,NaN,NaN,NaN,False
9,900,F,True,True,False,False,False,M,27-Nov-93,Black,...,WADM,INC,17-Oct-11,False,2.250,Y,0.0,0.0,0.0,False


In [218]:
print(list(df))
for col in df:
    print("Column:", col, "; Unique Values: ",df[col].unique())

['Term', 'Level', 'Prospect', 'Applicant', 'Admit', 'Deposit', 'Enroll', 'Gender', 'BIRTHDATE', 'Ethnicity', 'Region Description', 'Age', 'Age Category Description', 'ACT Category Description', 'ACTC', 'ACTE', 'ACTM', 'ACTR', 'ACTS', 'ACT_Test_Date', 'College', 'Major', 'Decision', 'GeneralDecision', 'PROG_ACTION', 'PROG_REASON', 'ApplicationDate', '1stGeneration', 'Convert_GPA_Fix', 'FIN_AID_INTEREST', 'Rank', 'Size', 'Percentile', 'Pell']
Column: Term ; Unique Values:  [900]
Column: Level ; Unique Values:  ['F' 'T' 'CN' 'M' 'S']
Column: Prospect ; Unique Values:  [ True]
Column: Applicant ; Unique Values:  [ True False]
Column: Admit ; Unique Values:  [False  True]
Column: Deposit ; Unique Values:  [False  True]
Column: Enroll ; Unique Values:  [False  True]
Column: Gender ; Unique Values:  ['F' 'U' 'M']
Column: BIRTHDATE ; Unique Values:  ['09-Jul-50' '28-Nov-88' '18-Jul-87' ... '22-Sep-95' '11-Feb-95'
 '13-Aug-95']
Column: Ethnicity ; Unique Values:  ['White' 'Multiracial' 'Hispani

In [219]:
df.isna().sum()

Term                           0
Level                          0
Prospect                       0
Applicant                      0
Admit                          0
Deposit                        0
Enroll                         0
Gender                         0
BIRTHDATE                      6
Ethnicity                      0
Region Description             0
Age                            6
Age Category Description       0
ACT Category Description       0
ACTC                        4991
ACTE                        4996
ACTM                        4996
ACTR                        4999
ACTS                        4999
ACT_Test_Date               4991
College                      105
Major                         98
Decision                      98
GeneralDecision               98
PROG_ACTION                   98
PROG_REASON                 1651
ApplicationDate               98
1stGeneration                  0
Convert_GPA_Fix             2174
FIN_AID_INTEREST              99
Rank      

In [220]:
#null_columns=df.columns[df.isnull().any()]
#print(df[df["Age"].isnull()][null_columns])
#print(df[df["Decision"].isnull()][null_columns])
print(df.loc[df['Enroll'] == True].count())
print(df.loc[(df['Admit'] == True) & (df['Enroll'] == False)].count())

Term                        2596
Level                       2596
Prospect                    2596
Applicant                   2596
Admit                       2596
Deposit                     2596
Enroll                      2596
Gender                      2596
BIRTHDATE                   2596
Ethnicity                   2596
Region Description          2596
Age                         2596
Age Category Description    2596
ACT Category Description    2596
ACTC                        2230
ACTE                        2230
ACTM                        2230
ACTR                        2230
ACTS                        2229
ACT_Test_Date               2230
College                     2596
Major                       2596
Decision                    2596
GeneralDecision             2596
PROG_ACTION                 2596
PROG_REASON                 1191
ApplicationDate             2596
1stGeneration               2596
Convert_GPA_Fix             2591
FIN_AID_INTEREST            2596
Rank      

In [221]:
hie = df.loc[df['Admit'] == True].copy()
print(hie.isna().sum())
hie.dropna(subset=['ACT_Test_Date'], inplace=True)
print(hie.isna().sum())

Term                           0
Level                          0
Prospect                       0
Applicant                      0
Admit                          0
Deposit                        0
Enroll                         0
Gender                         0
BIRTHDATE                      0
Ethnicity                      0
Region Description             0
Age                            0
Age Category Description       0
ACT Category Description       0
ACTC                        1878
ACTE                        1879
ACTM                        1880
ACTR                        1883
ACTS                        1883
ACT_Test_Date               1878
College                        6
Major                          0
Decision                       0
GeneralDecision                0
PROG_ACTION                    0
PROG_REASON                 1548
ApplicationDate                0
1stGeneration                  0
Convert_GPA_Fix               13
FIN_AID_INTEREST               0
Rank      

In [222]:
for col in hie:
    print("Column:", col, "; Unique Values: ",hie[col].unique())
hie.drop(columns=['Term','Prospect','Applicant','Admit','BIRTHDATE','Age Category Description', \
                  'ACT Category Description','PROG_REASON','PROG_ACTION','GeneralDecision'], inplace=True)      
print(hie.isna().sum())

Column: Term ; Unique Values:  [900]
Column: Level ; Unique Values:  ['F']
Column: Prospect ; Unique Values:  [ True]
Column: Applicant ; Unique Values:  [ True]
Column: Admit ; Unique Values:  [ True]
Column: Deposit ; Unique Values:  [ True False]
Column: Enroll ; Unique Values:  [ True False]
Column: Gender ; Unique Values:  ['M' 'F' 'U']
Column: BIRTHDATE ; Unique Values:  ['15-Nov-93' '31-Jul-94' '28-Jul-94' '27-May-94' '10-Dec-91' '03-Jul-94'
 '23-Feb-94' '11-Mar-94' '12-Nov-93' '05-Dec-92' '14-Mar-93' '05-Apr-94'
 '10-Jul-93' '17-Aug-93' '27-Oct-92' '27-Apr-94' '26-Nov-93' '27-Nov-93'
 '06-Jul-94' '10-Sep-93' '23-Feb-93' '03-Jan-94' '15-Apr-94' '29-Sep-93'
 '03-May-94' '20-Jun-94' '10-Jul-94' '24-Oct-93' '24-Mar-94' '15-Dec-93'
 '06-Sep-93' '17-Jan-94' '30-Oct-93' '02-Nov-94' '28-Dec-92' '16-Nov-92'
 '31-Mar-88' '20-Feb-93' '20-Aug-93' '07-Oct-94' '06-Jun-93' '17-Jul-93'
 '23-Jul-93' '17-Dec-93' '11-Jun-94' '02-Feb-94' '29-Aug-93' '24-Apr-94'
 '20-Sep-92' '27-Aug-93' '27-Nov-92'

Column: Rank ; Unique Values:  [139.   0.  35.   2.  20.  28.  53. 141. 126.  13.  94.  78.  40.  67.
  11.  32.  88.  22.   7.   9.  26.  29. 172. 105.  23.   3.  41. 171.
  39.  96.  44. 157.  14.  12.  10.  16. 357.  58.  91.  19.  75.  84.
 431.   1. 278.  24. 122.  54.  77. 257.  61.  36. 195.  57.  17. 190.
  18.  69.  31.  74. 177.  80.  85. 151.  68.  47.  49. 101.  51.   6.
  66. 316.  55. 125. 242. 334.  98.  46.   4. 180. 312. 176.  52.  90.
 184. 188.  42. 365. 142.  63. 162.  25. 400. 391.  93.  21. 246. 183.
  33. 112. 252. 110. 187. 202. 268.  60. 192. 206. 159. 100. 259.  50.
 203.  70.  64.  45.  27. 145.  73.  30. 503. 102. 103. 106.  62. 279.
 210. 127. 226. 107.  37. 200. 394. 135. 113.  15. 198.  59.  71.  34.
 201. 124.   5. 262. 232. 373. 108. 179. 133. 204. 136. 191.  43. 174.
 128. 115. 161. 118.  48. 367. 280. 173. 320.   8. 281. 233. 245. 148.
 163. 344.  79. 340. 165.  99. 381.  97. 298. 375. 399. 114. 194. 314.
  56. 225. 211. 220. 167. 386. 263.  65. 140. 

In [223]:
for col in hie:
    print("Column:", col, "; Unique Values: ",hie[col].unique())
print(hie.isna().sum())

Column: Level ; Unique Values:  ['F']
Column: Deposit ; Unique Values:  [ True False]
Column: Enroll ; Unique Values:  [ True False]
Column: Gender ; Unique Values:  ['M' 'F' 'U']
Column: Ethnicity ; Unique Values:  ['White' 'International' 'Hispanic' 'Asian/Pacific Islander'
 'Not Applicable' 'Black' 'Multiracial' 'American Indian/Alaskan Native'
 'Native Hawaiian/Oth Pac Island']
Column: Region Description ; Unique Values:  ['Suburban North' 'Chicago Central' 'Chicago South' 'Suburban West'
 'Chicago Far North' 'Out of State' 'Suburban Southwest' 'Other Illinois'
 'Suburban South' 'Chicago North' 'Chicago Far South' 'Suburban Northwest'
 'International' 'Northwest Indiana' 'Unknown']
Column: Age ; Unique Values:  [18. 20. 19. 17. 24. 23. 16. 22.  1. 27. 14. 21.]
Column: ACTC ; Unique Values:  [24. 23. 29. 27. 32. 36. 30. 21. 26. 28. 20. 19. 31. 18. 33. 22. 17. 34.
 25. 16. 15. 13. 14. 35.]
Column: ACTE ; Unique Values:  [25. 24. 28. 27. 34. 35. 22. 32. 23. 21. 31. 29. 30. 33. 20. 19.

Level                 0
Deposit               0
Enroll                0
Gender                0
Ethnicity             0
Region Description    0
Age                   0
ACTC                  0
ACTE                  1
ACTM                  2
ACTR                  5
ACTS                  5
ACT_Test_Date         0
College               0
Major                 0
Decision              0
ApplicationDate       0
1stGeneration         0
Convert_GPA_Fix       7
FIN_AID_INTEREST      0
Rank                  5
Size                  5
Percentile            5
Pell                  0
dtype: int64


In [224]:
hie['Days_Application_to_Start'] = pd.to_datetime('01-Sep-12') - pd.to_datetime(hie['ApplicationDate'])

In [225]:
#print(hie['Days_Application_to_Start'].str.slice(0, 3))
hie['Days_Application_to_Start'] = hie['Days_Application_to_Start'].astype(str).str.slice(0, 3).astype(int) % 365
hie['Days_Application_to_Start']

17       290
18       250
19       232
24       312
26       323
27       352
28       305
30       212
31       316
36       222
37       291
38       303
40       271
41       294
42       222
44       289
46       211
47       217
49       306
50       296
51       250
52       291
55       319
57       305
59       213
60         8
61       320
62       266
63       298
64       305
        ... 
17815    165
17825    159
17827     95
17831    165
17834    163
17838    163
17839    163
17841    162
17846    162
17847    159
17848    159
17861    157
17865    152
17868    155
17873    141
17875    152
17884    149
17886    152
17887    152
17892    151
17898    143
17908    145
17913    144
17922    138
17923    143
17954    134
17966    131
17967    131
18015    101
18062     10
Name: Days_Application_to_Start, Length: 9427, dtype: int32

In [226]:
hie['ACTE'].fillna(mode(hie.ACTE),inplace=True)
hie['ACTM'].fillna(mode(hie.ACTM),inplace=True)
hie['ACTR'].fillna(mode(hie.ACTR),inplace=True)
hie['ACTS'].fillna(mode(hie.ACTS),inplace=True)
hie['ACTC'].fillna(round(((hie.ACTE + hie.ACTM + hie.ACTR + hie.ACTS)/4),0),inplace=True)
hie['Convert_GPA_Fix'].fillna(mode(hie.Convert_GPA_Fix),inplace=True)
hie['Rank'].fillna(mode(hie.Rank),inplace=True)
hie['Size'].fillna(mode(hie.Size),inplace=True)
hie['Percentile'].fillna(mode(hie.Percentile),inplace=True)

In [227]:
hie = pd.get_dummies(data=hie,columns=['Gender','Decision','FIN_AID_INTEREST'])
hie.drop(columns=['Gender_M','Gender_U','Decision_WILLNOTENTER','FIN_AID_INTEREST_N'], inplace=True)
hie.dtypes

Level                         object
Deposit                         bool
Enroll                          bool
Ethnicity                     object
Region Description            object
Age                          float64
ACTC                         float64
ACTE                         float64
ACTM                         float64
ACTR                         float64
ACTS                         float64
ACT_Test_Date                 object
College                       object
Major                         object
ApplicationDate               object
1stGeneration                   bool
Convert_GPA_Fix              float64
Rank                         float64
Size                         float64
Percentile                   float64
Pell                            bool
Days_Application_to_Start      int32
Gender_F                       uint8
Decision_ACCEPT                uint8
FIN_AID_INTEREST_Y             uint8
dtype: object

In [228]:
print(hie.dtypes)
print("------------")
for col in hie:
    if (hie[col].dtype == 'bool'):
        hie[col] = hie[col].astype(int)
    elif (hie[col].dtype == 'object'):
        hie[col] = hie[col].astype('category')

Level                         object
Deposit                         bool
Enroll                          bool
Ethnicity                     object
Region Description            object
Age                          float64
ACTC                         float64
ACTE                         float64
ACTM                         float64
ACTR                         float64
ACTS                         float64
ACT_Test_Date                 object
College                       object
Major                         object
ApplicationDate               object
1stGeneration                   bool
Convert_GPA_Fix              float64
Rank                         float64
Size                         float64
Percentile                   float64
Pell                            bool
Days_Application_to_Start      int32
Gender_F                       uint8
Decision_ACCEPT                uint8
FIN_AID_INTEREST_Y             uint8
dtype: object
------------


In [229]:
Level = dict( enumerate(hie['Level'].cat.categories) )
Ethnicity = dict( enumerate(hie['Ethnicity'].cat.categories) )
Region_Description = dict( enumerate(hie['Region Description'].cat.categories) )
College = dict( enumerate(hie['College'].cat.categories) )
Major = dict( enumerate(hie['Major'].cat.categories) )

In [230]:
cat_columns = hie.select_dtypes(['category']).columns
hie[cat_columns] = hie[cat_columns].apply(lambda x: x.cat.codes)

In [231]:
for col in hie:
    print("Column:", col, "; Unique Values: ",hie[col].unique())

Column: Level ; Unique Values:  [0]
Column: Deposit ; Unique Values:  [1 0]
Column: Enroll ; Unique Values:  [1 0]
Column: Ethnicity ; Unique Values:  [8 4 3 1 7 2 5 0 6]
Column: Region Description ; Unique Values:  [ 9  0  4 13  1  8 12  7 11  3  2 10  5  6 14]
Column: Age ; Unique Values:  [18. 20. 19. 17. 24. 23. 16. 22.  1. 27. 14. 21.]
Column: ACTC ; Unique Values:  [24. 23. 29. 27. 32. 36. 30. 21. 26. 28. 20. 19. 31. 18. 33. 22. 17. 34.
 25. 16. 15. 13. 14. 35.]
Column: ACTE ; Unique Values:  [25. 24. 28. 27. 34. 35. 22. 32. 23. 21. 31. 29. 30. 33. 20. 19. 26. 16.
 17. 18. 36. 15. 13. 14.  9. 12. 11.  7. 10.  0.]
Column: ACTM ; Unique Values:  [19. 23. 20. 26. 32. 36. 28. 27. 15. 31. 25. 16. 30. 29. 22. 18. 24. 34.
 35. 21. 33. 17. 14.  6.  3. 12.]
Column: ACTR ; Unique Values:  [27. 22. 23. 36. 29. 31. 24. 32. 26. 19. 28. 34. 20. 18. 33. 17. 21. 15.
 35. 30. 25. 16. 13. 14. 12.  7. 10. 11.  5.  2.]
Column: ACTS ; Unique Values:  [24. 25. 23. 30. 35. 21. 27. 29. 28. 17. 20. 32. 1

In [232]:
# optional columns to drop
hie.drop(columns=['ApplicationDate','ACT_Test_Date','Decision_ACCEPT'], inplace=True)

In [233]:
hie.isnull().sum()

Level                        0
Deposit                      0
Enroll                       0
Ethnicity                    0
Region Description           0
Age                          0
ACTC                         0
ACTE                         0
ACTM                         0
ACTR                         0
ACTS                         0
College                      0
Major                        0
1stGeneration                0
Convert_GPA_Fix              0
Rank                         0
Size                         0
Percentile                   0
Pell                         0
Days_Application_to_Start    0
Gender_F                     0
FIN_AID_INTEREST_Y           0
dtype: int64

In [234]:
enroll = hie.loc[hie['Enroll'] == True].copy()
no_enroll = hie.loc[hie['Enroll'] == False].copy()
enroll.drop(columns=["Enroll"], inplace=True)
no_enroll.drop(columns=["Enroll"], inplace=True)

In [249]:
#transform into numpy array
x = enroll.reset_index().values

en_km = kmodes.KModes(n_clusters=5, init='Huang', n_init=5, verbose=0)
clusters = en_km.fit_predict(x)
#hie['clusters'] = clusters

# Print the cluster centroids
print(en_km.cluster_centroids_)


[[ 37.    0.    1.    8.    8.   18.   27.   24.   27.   24.   25.    0.
    0.    0.    4.    0.    0.    0.    0.  291.    0.    1. ]
 [ 17.    0.    1.    8.    8.   18.   25.   25.   26.   23.   24.    4.
  102.    0.    4.    0.    0.    0.    0.  292.    1.    1. ]
 [ 40.    0.    1.    3.   13.   18.   22.   23.   23.   20.   21.    6.
   83.    1.    3.5   0.    0.    0.    1.  291.    1.    1. ]]


In [236]:
#transform into numpy array
x = no_enroll.reset_index().values

noen_km = kmodes.KModes(n_clusters=5, init='Huang', n_init=5, verbose=0)
clusters = noen_km.fit_predict(x)
#hie['clusters'] = clusters

# Print the cluster centroids
print(noen_km.cluster_centroids_)

[[ 44.   0.   0.   8.   8.  18.  23.  23.  23.  24.  22.   6.  83.   0.
    4.   0.   0.   0.   0. 289.   1.   0.]
 [ 19.   0.   0.   8.   8.  18.  23.  25.  24.  24.  23.   0.  12.   0.
    4.   0.   0.   0.   0. 289.   0.   1.]
 [ 28.   0.   0.   8.   8.  18.  22.  22.  22.  22.  24.   4. 102.   0.
    4.   0.   0.   0.   0. 289.   1.   1.]
 [ 18.   0.   0.   3.   8.  18.  25.  24.  26.  26.  25.   6.  11.   0.
    4.   0.   0.   0.   0. 290.   0.   1.]
 [ 36.   0.   0.   3.   3.  18.  23.  23.  25.  20.  21.   6.  11.   1.
    4.   3. 986.  99.   1. 289.   1.   1.]]


In [250]:
headers = list(enroll)
headers = ['Column0'] + headers
headers

['Column0',
 'Level',
 'Deposit',
 'Ethnicity',
 'Region Description',
 'Age',
 'ACTC',
 'ACTE',
 'ACTM',
 'ACTR',
 'ACTS',
 'College',
 'Major',
 '1stGeneration',
 'Convert_GPA_Fix',
 'Rank',
 'Size',
 'Percentile',
 'Pell',
 'Days_Application_to_Start',
 'Gender_F',
 'FIN_AID_INTEREST_Y']

In [251]:
en_df = pd.DataFrame(en_km.cluster_centroids_, columns = headers)
noen_df = pd.DataFrame(noen_km.cluster_centroids_, columns = headers)

In [252]:
en_df["Level"].replace(Level, inplace=True)
en_df["Ethnicity"].replace(Ethnicity, inplace=True)
en_df["Region Description"].replace(Region_Description, inplace=True)
en_df["College"].replace(College, inplace=True)
en_df["Major"].replace(Major, inplace=True)

In [253]:
noen_df["Level"].replace(Level, inplace=True)
noen_df["Ethnicity"].replace(Ethnicity, inplace=True)
noen_df["Region Description"].replace(Region_Description, inplace=True)
noen_df["College"].replace(College, inplace=True)
noen_df["Major"].replace(Major, inplace=True)

In [254]:
first = list( ['Level','Ethnicity','Region Description','Age','College','Major','Gender_F','1stGeneration'])
second = list( ['ACTC','ACTE','ACTM','ACTR','ACTS','Convert_GPA_Fix','Rank','Size','Percentile','Days_Application_to_Start','Pell', \
                'Deposit','FIN_AID_INTEREST_Y'])

In [255]:
print(en_df[first])

  Level Ethnicity Region Description   Age                         College  \
0     F     White       Out of State  18.0                        Commerce   
1     F     White       Out of State  18.0  Liberal Arts & Social Sciences   
2     F  Hispanic      Suburban West  18.0              Science and Health   

                 Major  Gender_F  1stGeneration  
0          Accountancy       0.0            0.0  
1  UGRD Undeclared TBD       1.0            0.0  
2           Psychology       1.0            1.0  


In [256]:
print(en_df[second])

   ACTC  ACTE  ACTM  ACTR  ACTS  Convert_GPA_Fix  Rank  Size  Percentile  \
0  27.0  24.0  27.0  24.0  25.0              4.0   0.0   0.0         0.0   
1  25.0  25.0  26.0  23.0  24.0              4.0   0.0   0.0         0.0   
2  22.0  23.0  23.0  20.0  21.0              3.5   0.0   0.0         0.0   

   Days_Application_to_Start  Pell  Deposit  FIN_AID_INTEREST_Y  
0                      291.0   0.0      1.0                 1.0  
1                      292.0   0.0      1.0                 1.0  
2                      291.0   1.0      1.0                 1.0  


In [244]:
print(noen_df[first])

  Level Ethnicity Region Description   Age                         College  \
0     F     White       Out of State  18.0              Science and Health   
1     F     White       Out of State  18.0                        Commerce   
2     F     White       Out of State  18.0  Liberal Arts & Social Sciences   
3     F  Hispanic       Out of State  18.0              Science and Health   
4     F  Hispanic      Chicago North  18.0              Science and Health   

                     Major  Gender_F  1stGeneration  
0               Psychology       1.0            0.0  
1  Business Administration       0.0            0.0  
2      UGRD Undeclared TBD       1.0            0.0  
3      Biological Sciences       0.0            0.0  
4      Biological Sciences       1.0            1.0  


In [245]:
print(noen_df[second])

   ACTC  ACTE  ACTM  ACTR  ACTS  Convert_GPA_Fix  Rank   Size  Percentile  \
0  23.0  23.0  23.0  24.0  22.0              4.0   0.0    0.0         0.0   
1  23.0  25.0  24.0  24.0  23.0              4.0   0.0    0.0         0.0   
2  22.0  22.0  22.0  22.0  24.0              4.0   0.0    0.0         0.0   
3  25.0  24.0  26.0  26.0  25.0              4.0   0.0    0.0         0.0   
4  23.0  23.0  25.0  20.0  21.0              4.0   3.0  986.0        99.0   

   Days_Application_to_Start  Pell  Deposit  FIN_AID_INTEREST_Y  
0                      289.0   0.0      0.0                 0.0  
1                      289.0   0.0      0.0                 1.0  
2                      289.0   0.0      0.0                 1.0  
3                      290.0   0.0      0.0                 1.0  
4                      289.0   1.0      0.0                 1.0  


In [248]:
print(df.count())
print(hie.count())
print(enroll.count())
print(no_enroll.count())

Term                        18197
Level                       18197
Prospect                    18197
Applicant                   18197
Admit                       18197
Deposit                     18197
Enroll                      18197
Gender                      18197
BIRTHDATE                   18191
Ethnicity                   18197
Region Description          18197
Age                         18191
Age Category Description    18197
ACT Category Description    18197
ACTC                        13206
ACTE                        13201
ACTM                        13201
ACTR                        13198
ACTS                        13198
ACT_Test_Date               13206
College                     18092
Major                       18099
Decision                    18099
GeneralDecision             18099
PROG_ACTION                 18099
PROG_REASON                 16546
ApplicationDate             18099
1stGeneration               18197
Convert_GPA_Fix             16023
FIN_AID_INTERE